In [2]:
!pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0

  Using cached tensorflow_serving_api-2.7.0-py2.py3-none-any.whl (37 kB)
  Using cached libclang-14.0.6-py2.py3-none-macosx_10_9_x86_64.whl (13.2 MB)
  Using cached protobuf-3.19.6-cp39-cp39-macosx_10_9_x86_64.whl (980 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached tensorflow_io_gcs_filesystem-0.28.0-cp39-cp39-macosx_10_14_x86_64.whl (1.6 MB)
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 20210226132247
    Uninstalling flatbuffers-20210226132247:
      Successfully uninstalled flatbuffers-20210226132247


  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.1
    Uninstalling protobuf-3.20.1:
      Successfully uninstalled protobuf-3.20.1


In [3]:
!pip install keras-image-helper

  Using cached keras_image_helper-0.0.1-py3-none-any.whl (4.6 kB)
  Using cached Pillow-9.3.0-cp39-cp39-macosx_10_10_x86_64.whl (3.3 MB)


In [4]:
import grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [5]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [6]:
from keras_image_helper import create_preprocessor

In [7]:
preprocessor = create_preprocessor('xception', target_size=(299,299))

In [8]:
# plate example 
url = 'https://raw.githubusercontent.com/mary435/kitchenware_classification/main/images/6172.jpg'
X = preprocessor.from_url(url)

In [10]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [11]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'kitchenware-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_45'].CopyFrom(np_to_protobuf(X))


In [12]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [13]:
preds = pb_response.outputs['dense_35'].float_val

In [14]:
classes = ['cup', 'fork', 'glass', 'knife', 'plate', 'spoon']

In [15]:
dict(zip(classes, preds))

{'cup': -0.4962674379348755,
 'fork': -20.078845977783203,
 'glass': -1.2485648393630981,
 'knife': -6.5643463134765625,
 'plate': 25.220046997070312,
 'spoon': -23.530792236328125}

In [17]:
result = dict(zip(preds, classes))
result[max(result)]

'plate'